<a href="https://colab.research.google.com/github/Gaussiandra/allcups_2020_3/blob/master/0_3767_2019_06_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-d96b70cc-90bd-9d90-7583-7f1fb0d71fdf)


In [1]:
!pip install catboost
from catboost import CatBoostClassifier

In [2]:
!pip install lightfm
from lightfm import LightFM

     |████████████████████████████████| 307kB 2.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707617 sha256=0f4a772ffa16e129ea62589a889cb21d128fbbd2c1e25f055dfe7a10b6bac815
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import json
import pickle
import re
import nltk

import lightgbm as lgb

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

In [4]:
tqdm.pandas(tqdm)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

global_path = '/content/drive/My Drive/ML/Выявление закономерностей на основе данных МФЦ/'
irkutsk_ds_path = global_path + 'data/dataset_irkutsk/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
categorical_features = [
    #'requester',
    'service',  
    'service_title', 
    'receipt_mfc',
    'internal_status', 
    'external_status',
    'sub_department',  
    'creation_mfc', 
    'order_type', 
    'department_id',
    'deleted', 
    'deleter_fk', 
    'custom_service_id', 
    'service_level',
    'mfc',
    'cpgu_user',
    'text_cluster',
    'emb_item_cluster',

    #'nan_added_col',  
    'previous_service_title',
    'previous_text_cluster',    
    'requester_requester_type', 
    'requester_gender',
    'requester_deleted',
    
    #'cpgu_department_id', 
    'cpgu_department_parent', 
    'cpgu_department_code',
    'cpgu_department_calendar',
    'cpgu_department_department_id',
    'cpgu_department_cluster',

    'cpgu_mfc_id',
    'cpgu_mfc_code', 
    'cpgu_mfc_closed', 
    'cpgu_mfc_branch_id',
    'cpgu_mfc_qms_server_id',
    'cpgu_mfc_deperatment_id', 
    'cpgu_mfc_office_type_id',
    'cpgu_mfc_parent_id', 
    'cpgu_mfc_key_mfc', 
    'cpgu_mfc_calendar',
    'cpgu_mfc_damask_office_id',

    'cpgu_user_auto_ping_queue',

    'cpgu_service_level',
    'cpgu_service_is_not_render',
    'cpgu_service_person',
    'cpgu_service_sole',
    'cpgu_service_legal',

    'custom_service_info_person',
    'custom_service_info_sole',
    'custom_service_info_legal',	
    'custom_service_info_group_title',
    'custom_service_info_department',	
    'custom_service_info_level',	
    'custom_service_info_is_not_render',
    'custom_service_info_deleted',	
    #'custom_service_info_frgu_service_id',

    #'user_to_emb_cluster'
]

# Import data

In [7]:
# Добавить считывание без ошибок и с проставленными типами

csv_tables = {}
for file_name in tqdm(os.listdir(irkutsk_ds_path)[:-6]):
    csv_tables[file_name.replace('.csv', '')] = pd.read_csv(
        irkutsk_ds_path+file_name,
        encoding='cp1251',
        sep=';',
        error_bad_lines=False,
    )

len(csv_tables)

b'Skipping line 5352: expected 5 fields, saw 8\nSkipping line 9698: expected 5 fields, saw 8\nSkipping line 10007: expected 5 fields, saw 8\nSkipping line 10352: expected 5 fields, saw 8\nSkipping line 11724: expected 5 fields, saw 8\nSkipping line 11831: expected 5 fields, saw 8\nSkipping line 27343: expected 5 fields, saw 8\nSkipping line 29624: expected 5 fields, saw 8\n'
b'Skipping line 2128421: expected 5 fields, saw 8\nSkipping line 2128802: expected 5 fields, saw 8\nSkipping line 2130406: expected 5 fields, saw 8\nSkipping line 2131359: expected 5 fields, saw 8\nSkipping line 2132539: expected 5 fields, saw 8\nSkipping line 2143803: expected 5 fields, saw 8\nSkipping line 2153411: expected 5 fields, saw 8\nSkipping line 2159798: expected 5 fields, saw 8\nSkipping line 2175361: expected 5 fields, saw 8\nSkipping line 2175381: expected 5 fields, saw 8\nSkipping line 2180132: expected 5 fields, saw 8\nSkipping line 2181389: expected 5 fields, saw 8\nSkipping line 2187745: expected 

17

In [8]:
train_csv = pd.read_csv(irkutsk_ds_path+'train.csv')

train_csv.columns = train_csv.iloc[-1]
train_csv.drop(2713722, inplace=True)
train_csv.columns.name = None

# Написать в загрузку
train_csv.requester = train_csv.requester.astype(int)
train_csv.service_title = train_csv.service_title.astype(int)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
def handle_type(column):
    try:
        return column.astype(int)
    except:
        try:
            return column.astype(float)
        except:
            print(column.name)
            return column

In [10]:
for name, table in tqdm(csv_tables.items()):
    print('!'+name)
    for col in table.columns:
        table[col] = handle_type(table[col])

!cpgu_department
department_id
code
title
!cpgu_mfc
name
full_name
address
key_mfc
!cpgu_service
description
level
eid
lid
!cpgu_sub_department
title
!cpgu_user
!custom_service_info
title
group_title
level
eid
lid
parent_service_eid
parent_service_lid
!enum_external_order_status
external_order_status
localized_status_name
!enum_order_status
localized_status_name
order_status
!equeue_ticket
id
ticket_number
serving_started
serving_finished
service_name
!equeue_ticket_order_act
equeue_ticket
type
change_timestamp
!MaleFemale
Name
!order_relates
!order_status_history
Last_status_time
!requester
edit_timestamp
!RequesterType
Name
!department
name
!office_type
name



In [11]:
test_ids_txt = open(irkutsk_ds_path+'test_ids.txt', 'r').read().split('\n')
test_ids = pd.Series(test_ids_txt[1:-1], name=test_ids_txt[0], dtype=int)
test_ids

/usr/local/lib/python3.6/dist-packages/numpy/core/numeric.py:2327: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


0         83029018
1        123998836
2        184301121
3         54344611
4        184299902
           ...    
44439     27213855
44440     64899947
44441     23695212
44442     72707269
44443     38205629
Name: requester, Length: 44444, dtype: int64

In [12]:
service_title_mapping_json = json.load(open(irkutsk_ds_path+'service_title_mapping.json', 'r'))
service_title_mapping_dict = {}

for item in service_title_mapping_json.items():
    service_title_mapping_dict[item[1]] = item[0]

service_title_mapping_dict[0], len(service_title_mapping_dict)

('_Присвоение (аннулирование) адреса объекту адресации', 1310)

In [13]:
id_to_desc = pd.DataFrame.from_records(list(service_title_mapping_dict.items())).drop(0, axis=1)
id_to_desc

,1
0,_Присвоение (аннулирование) адреса объекту адр...
1,Принятие документов для принятия решения о под...
2,_Выдача разрешений на ввод объекта в эксплуата...
3,"Выдача справок о том, является или не является..."
4,"_Предоставление сведений, содержащихся в Едино..."
...,...
1305,Прием расчета по начисленным и уплаченным стра...
1306,Услуга по предоставлению по заданным параметра...
1307,_Принятие решения о предоставлении ветеранам т...
1308,Прием заявления физического лица (его законног...


# Preprocessing

In [14]:
# скорее всего нам нужно брать только первое посещение, а это 900к примеров и мы можем даже уместиться 
# Тогда мы не отслеживаем историю и делаем фичи только для пользователей из теста
# А что если этот пользователь уже сущестует, просто ему изменили id? :thinking:
# Время ожидания, день недели и месяца, день года в праздник, час
# Выбросы
#   предсказывать категорию и давать самый популярный ответ
# Часовые пояса
#   время с последней транзакции
# Проверить сортировку
# Геокодинг?
# Нужно ли выделять выходные при feature encoding в catboost
#   Информация о предыдущих заявлениях(больше чем сдвиг на 1), их количество!!!!
# Bag of words на заявлениях и количествах, время с последнего пользования в каждой категории, нграмы, эмбеддинги?((( + общее колво
# Аугментация
#   кластеризовать cpu service description, весь дф, pca, ica
# Просто соединить всё в одну таблицу в т.ч. пол, возвраст
# Буква из ticket name
# Pipeline 
# кластеризовать текст в service_title_mapping_dict или срацу пользователей и мфц с помощью bpr/lightfm
# Можно предсказывать сначала кластеризированную категорию, а потом уже сам сервис
# Как формировать тест?
# Разрыв во времени при формировании таргета?
# id
# Количество посещенных мфц
# Вроде залив просто трейна даст 0%, значит можно делать проверку предсказаний
# Карантин
# Проверить типы, 
# МФЦ/ТОСП?
# Время ожидания ?
# bpr, lightfm, pca, ica
# остаток от деления на 1000 в timestamp
#   частота посещений
# подчеркивания в названии классов
#   больше сдвиг и prev_service_title
# ковырять тексты
# json
# больше информации о service_title
# последовательности 
# зависимость времени выгрузки
# информация из файлов
# Больше кластеризаций, больше данных
#   карантин: данные только после него, фича на кол-во посещений до и после, карантин ли в данный момент
# посмотреть даты для теста
# как у него 90% 41+ дают. проверить классы

#### user/item embeddings

In [15]:
last_visits_id = train_csv.reset_index().groupby('requester')['index'].last()

In [16]:
pivot_df = (
    train_csv.drop(last_visits_id.values)
    .reset_index()
    .groupby(['requester', 'service_title'])['index']
    .count()
).reset_index(level=[0, 1])

In [17]:
pivot_df['index'] = np.log(pivot_df['index']) + 1

In [18]:
missing_requesters = set(train_csv.requester.values) - set(pivot_df.requester.values)
missing_requesters_df = pd.DataFrame({
    'requester': list(missing_requesters), 
    'service_title': 4,
    'index': 0
})

In [19]:
pivot_df = pivot_df.append(missing_requesters_df, ignore_index=True)

In [20]:
pivot_df

,requester,service_title,index
0,1651,116,1.0
1,1651,339,1.0
2,1651,1169,1.0
3,1651,1246,1.0
4,1651,1256,1.0
...,...,...,...
1385828,85983217,4,0.0
1385829,158334963,4,0.0
1385830,17825779,4,0.0
1385831,67108858,4,0.0


In [21]:
# https://stackoverflow.com/a/53235048/12495874

from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

person_c = CategoricalDtype(sorted(pivot_df.requester.unique()), ordered=True)
thing_c = CategoricalDtype(sorted(pivot_df.service_title.unique()), ordered=True)

row = pivot_df.requester.astype(person_c).cat.codes
col = pivot_df.service_title.astype(thing_c).cat.codes

sparse_matrix = csr_matrix(
    (pivot_df['index'], (row, col)),
    shape=(person_c.categories.size, thing_c.categories.size)
)

In [ ]:
lfm_model = LightFM(
    no_components=64,
    loss='warp',
    learning_rate=0.05,
    random_state=1234)

lfm_model.fit(sparse_matrix, epochs=100, num_threads=2, verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [ ]:
#pickle.dump(lfm_model, open(global_path+'lightfm_model', 'wb'))

In [22]:
lfm_model = pickle.load(open(global_path+'lightfm_model', 'rb'))

In [ ]:
# lfm_preds = lfm_model.user_embeddings.dot(lfm_model.item_embeddings.T)
# lfm_preds_df = pd.DataFrame(lfm_preds, person_c.categories.values)
# lfm_preds_df.columns = ['lfm_pred_'+str(col) for col in lfm_preds_df.columns]
# lfm_preds_df

In [ ]:
# user_embeddings_df = pd.DataFrame(lfm_model.user_embeddings, person_c.categories.values)
# user_embeddings_df.columns = ['user_embedding_'+str(col) for col in user_embeddings_df.columns]
# user_embeddings_df

In [23]:
clustered_item_emb = AgglomerativeClustering(
    n_clusters=None,
    linkage='average',
    distance_threshold=6
).fit_predict(lfm_model.item_embeddings)

len(np.unique(clustered_item_emb)), silhouette_score(lfm_model.item_embeddings, clustered_item_emb)

(93, 0.19057237)

In [24]:
emb_item_cluster = pd.DataFrame(
    clustered_item_emb, 
    thing_c.categories, 
    ['emb_item_cluster']
)

In [25]:
emb_item_cluster

,emb_item_cluster
0,0
1,0
2,0
3,0
4,51
...,...
1304,1
1305,0
1306,0
1307,32


In [26]:
train_csv = train_csv.merge(
    emb_item_cluster.reset_index(),
    how='left',
    left_on='service_title',
    right_on='index'
).drop('index', axis=1)

In [27]:
train_csv

,order_date,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,close_date,service_level,issue_date,change_timestamp,emb_item_cluster
0,2019-01-06 16:42:30.203000000,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-06,FEDERAL,2019-01-06,2019-01-06 16:49:07.493000000,90.0
1,2019-01-06 17:40:47.163000000,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298,2019-01-06,FEDERAL,2019-01-06,2019-01-06 17:42:28.270000000,0.0
2,2019-01-08 14:24:48.943000000,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,4.83333e+07,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:17:09.787000000,0.0
3,2019-01-08 15:19:51.693000000,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:20:31.417000000,90.0
4,2019-01-08 15:28:17.503000000,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:29:53.653000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713717,2020-05-18 19:06:26.653000000,177596976,43,306416246,1275,4396466,8377002,4396466,2,2,63807360,4396466,SERVICE,12870937,0,NaN,210963,NaN,FEDERAL,NaN,2020-05-18 19:12:31.830000000,6.0
2713718,2020-05-18 19:07:50.170000000,3663125,43,79455276,98,1956656,8377003,1956656,2,2,NaN,1956656,SERVICE,316134,0,NaN,48333290,NaN,FEDERAL,NaN,2020-05-18 19:17:28.360000000,87.0
2713719,2020-05-18 19:26:17.127000000,86143570,43,48742042,98,19146184,8377004,19146184,2,2,NaN,19146184,SERVICE,316134,0,NaN,48333290,NaN,FEDERAL,NaN,2020-05-18 19:27:34.843000000,87.0
2713720,2020-05-18 19:32:53.107000000,138811055,43,1970511,491,1956423,8377005,1956423,10,5,NaN,1956423,SERVICE,316160,0,NaN,210889936,2020-05-18,FEDERAL,2020-05-18,2020-05-18 19:33:49.607000000,90.0


#### data cleaning

In [28]:
issue_bad_ids = train_csv['issue_date'][train_csv['issue_date'].isin([
    '86538948'
])].index.tolist()

ct_bad_ids = train_csv['change_timestamp'][train_csv['change_timestamp'].isin([
    'MUNICIPAL', 
    '47208896', 
    'REGIONAL'
])].index.values.tolist()

on_bad_ids = train_csv['order_number'][train_csv['order_number'].isin([
    '6930981-46010-6930981'
])].index.values.tolist()

In [29]:
bad_ids = list(set().union(issue_bad_ids+ct_bad_ids+on_bad_ids))
print(len(bad_ids))

train_csv = train_csv.drop(bad_ids).reset_index().drop('index', axis=1)

277


In [30]:
print('!train')
for col in train_csv.columns:
    train_csv[col] = handle_type(train_csv[col])

!train
order_date
order_type
close_date
service_level
issue_date
change_timestamp


---

In [31]:
n_visits = train_csv.groupby('requester')['service_title'].count().rename('n_visits')

In [32]:
train_csv = train_csv.merge(n_visits, how='left', left_on='requester', right_on='requester')

#### json

In [33]:
def token_and_stem(text):
    def handle_clown_string(string):
        return re.sub(r'[\W_]', ' ', string.strip()).strip()

    stemmer = nltk.stem.snowball.SnowballStemmer("russian")

    tokens = [word for sent in nltk.sent_tokenize(handle_clown_string(text)) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [34]:
tfidf_vect = TfidfVectorizer(tokenizer=token_and_stem).fit_transform(id_to_desc[1].values)

In [35]:
agg_clustesters = AgglomerativeClustering(n_clusters=320, affinity='euclidean').fit_predict(tfidf_vect.toarray())

In [36]:
id_to_desc['text_cluster'] = agg_clustesters

In [37]:
grouped_strings = id_to_desc.groupby('text_cluster')[1].apply(list)

for ind, group in enumerate(sorted(grouped_strings)):
    print('___', ind)
    for i in sorted(group):
        print(i)

___ 0
_Бесплатное информирование (в том числе в письменной форме) налогоплательщиков, плательщиков сборов, плательщиков страховых взносов и налоговых агентов о действующих налогах и сборах, страховых взносах, законодательстве Российской Федерации о налогах и сборах и принятых в соответствии с ним нормативных правовых актах, порядке исчисления и уплаты налогов и сборов, страховых взносов, правах и обязанностях налогоплательщиков, плательщиков сборов, плательщиков страховых взносов и налоговых агентов, полномочиях налоговых органов и их должностных лиц (в части приема запроса и выдачи справки об исполнении налогоплательщиком (плательщиком сборов, плательщиком страховых взносов, налоговым агентом) обязанности по уплате налогов, сборов, страховых взносов, пеней, штрафов, процентов)
_Бесплатное информирование (в том числе в письменной форме) налогоплательщиков, плательщиков сборов, плательщиков страховых взносов и налоговых агентов о действующих налогах и сборах, страховых взносах, законода

In [38]:
id_to_desc

,1,text_cluster
0,_Присвоение (аннулирование) адреса объекту адр...,132
1,Принятие документов для принятия решения о под...,183
2,_Выдача разрешений на ввод объекта в эксплуата...,95
3,"Выдача справок о том, является или не является...",311
4,"_Предоставление сведений, содержащихся в Едино...",39
...,...,...
1305,Прием расчета по начисленным и уплаченным стра...,246
1306,Услуга по предоставлению по заданным параметра...,68
1307,_Принятие решения о предоставлении ветеранам т...,182
1308,Прием заявления физического лица (его законног...,277


In [39]:
train_csv = train_csv.merge(id_to_desc['text_cluster'].reset_index(), how='left', left_on='service_title', right_on='index')

In [40]:
train_csv.drop('index', axis=1, inplace=True)

In [41]:
train_csv

,order_date,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,close_date,service_level,issue_date,change_timestamp,emb_item_cluster,n_visits,text_cluster
0,2019-01-06 16:42:30.203000000,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-06,FEDERAL,2019-01-06,2019-01-06 16:49:07.493000000,90.0,150,282
1,2019-01-06 17:40:47.163000000,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,2019-01-06,FEDERAL,2019-01-06,2019-01-06 17:42:28.270000000,0.0,3,181
2,2019-01-08 14:24:48.943000000,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,48333290.0,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:17:09.787000000,0.0,150,315
3,2019-01-08 15:19:51.693000000,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:20:31.417000000,90.0,150,282
4,2019-01-08 15:28:17.503000000,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,2019-01-08,FEDERAL,2019-01-08,2019-01-08 15:29:53.653000000,0.0,3,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713440,2020-05-18 19:06:26.653000000,177596976,43,306416246,1275,4396466,8377002,4396466,2,2,63807360.0,4396466,SERVICE,12870937,0,NaN,210963.0,NaN,FEDERAL,NaN,2020-05-18 19:12:31.830000000,6.0,8,104
2713441,2020-05-18 19:07:50.170000000,3663125,43,79455276,98,1956656,8377003,1956656,2,2,NaN,1956656,SERVICE,316134,0,NaN,48333290.0,NaN,FEDERAL,NaN,2020-05-18 19:17:28.360000000,87.0,7,315
2713442,2020-05-18 19:26:17.127000000,86143570,43,48742042,98,19146184,8377004,19146184,2,2,NaN,19146184,SERVICE,316134,0,NaN,48333290.0,NaN,FEDERAL,NaN,2020-05-18 19:27:34.843000000,87.0,2,315
2713443,2020-05-18 19:32:53.107000000,138811055,43,1970511,491,1956423,8377005,1956423,10,5,NaN,1956423,SERVICE,316160,0,NaN,210889936.0,2020-05-18,FEDERAL,2020-05-18,2020-05-18 19:33:49.607000000,90.0,3,282


#### datetime

In [42]:
last_datetime = pd.to_datetime(train_csv['order_date'].max())

In [43]:
def preprocess_datetime(column, last_time=last_datetime):
    dt_column = column.astype('datetime64')
    
    final_df = pd.concat([
        dt_column.dt.year.rename('year'),
        dt_column.dt.dayofyear.rename('day_of_year'),
        dt_column.dt.month.rename('month'),
        dt_column.dt.weekday.rename('weekday'),
        dt_column.dt.day.rename('day'),
        dt_column.dt.hour.rename('hour'),
        dt_column.dt.minute.rename('minute'),

        (last_time - dt_column).dt.total_seconds().rename('ms_since_last_time'),
    ], axis=1)
    final_df.columns = [f'{column.name}_{col}' for col in final_df.columns]

    return final_df

In [44]:
prep_time_columns = ['order_date', 'issue_date', 'change_timestamp', 'close_date']

In [45]:
prep_time_df = train_csv[prep_time_columns].apply(pd.to_datetime)
prep_time_df_shifted = train_csv.groupby('requester').shift()[prep_time_columns].apply(pd.to_datetime)

time_since_last_act = (prep_time_df - prep_time_df_shifted).apply(lambda x: x.dt.total_seconds())

time_since_last_act.columns = ['time_since_last_act_'+col for col in time_since_last_act.columns]

In [46]:
train_csv_datetimes = pd.concat([
    time_since_last_act,
    *[preprocess_datetime(train_csv[col]) for col in prep_time_columns]
], axis=1)

In [47]:
train_csv_datetimes['waiting_time_ord_cls'] = train_csv_datetimes['order_date_ms_since_last_time'] - train_csv_datetimes['close_date_ms_since_last_time']

In [48]:
train_csv.order_date = train_csv.order_date.astype('datetime64[ns]')

train_od_by_req = train_csv.groupby('requester')['order_date']

visit_freq = (
    train_od_by_req.max() - 
    train_od_by_req.min()
 ) / n_visits

visit_freq = visit_freq.dt.total_seconds().rename('visit_freq')

In [49]:
train_csv = train_csv.merge(visit_freq, how='left', left_on='requester', right_on='requester')

In [50]:
train_csv.drop(prep_time_columns, axis=1, inplace=True)

In [51]:
train_csv_datetimes

,time_since_last_act_order_date,time_since_last_act_issue_date,time_since_last_act_change_timestamp,time_since_last_act_close_date,order_date_year,order_date_day_of_year,order_date_month,order_date_weekday,order_date_day,order_date_hour,order_date_minute,order_date_ms_since_last_time,issue_date_year,issue_date_day_of_year,issue_date_month,issue_date_weekday,issue_date_day,issue_date_hour,issue_date_minute,issue_date_ms_since_last_time,change_timestamp_year,change_timestamp_day_of_year,change_timestamp_month,change_timestamp_weekday,change_timestamp_day,change_timestamp_hour,change_timestamp_minute,change_timestamp_ms_since_last_time,close_date_year,close_date_day_of_year,close_date_month,close_date_weekday,close_date_day,close_date_hour,close_date_minute,close_date_ms_since_last_time,waiting_time_ord_cls
0,NaN,NaN,NaN,NaN,2019,6,1,6,6,16,42,4.303757e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,2019,6,1,6,6,16,49,4.303718e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,-60150.203
1,NaN,NaN,NaN,NaN,2019,6,1,6,6,17,40,4.303408e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,2019,6,1,6,6,17,42,4.303398e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,-63647.163
2,1.645387e+05,172800.0,1.672823e+05,172800.0,2019,8,1,1,8,14,24,4.287304e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,17,4.286990e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-51888.943
3,3.302750e+03,0.0,2.016300e+02,0.0,2019,8,1,1,8,15,19,4.286973e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,20,4.286969e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-55191.693
4,1.648503e+05,172800.0,1.648454e+05,172800.0,2019,8,1,1,8,15,28,4.286923e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,29,4.286913e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-55697.503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713440,3.538160e+03,NaN,3.366123e+03,NaN,2020,139,5,0,18,19,6,1.738430e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,139,5,0,18,19,12,1.373253e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2713441,5.634070e+02,NaN,-1.889000e+01,NaN,2020,139,5,0,18,19,7,1.654913e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,139,5,0,18,19,17,1.076723e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2713442,3.086784e+04,NaN,1.849730e+02,NaN,2020,139,5,0,18,19,26,5.479560e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,139,5,0,18,19,27,4.702400e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2713443,1.912412e+07,18403200.0,1.843548e+07,18403200.0,2020,139,5,0,18,19,32,1.519760e+02,2020.0,139.0,5.0,0.0,18.0,0.0,0.0,7.052508e+04,2020,139,5,0,18,19,33,9.547600e+01,2020.0,139.0,5.0,0.0,18.0,0.0,0.0,7.052508e+04,-70373.107


#### prev/next service_title

In [52]:
next_shifted_st = train_csv.groupby(['requester'])[['service_title', 'text_cluster']].shift()
next_shifted_st.columns = ['previous_service_title', 'previous_text_cluster']

target = train_csv.groupby(['requester'])['service_title'].shift(-1).rename('target')

#### requester

In [53]:
csv_tables['requester'] = pd.concat([
    csv_tables['requester'],
    preprocess_datetime(csv_tables['requester']['edit_timestamp'])
], axis=1)

csv_tables['requester'].drop('edit_timestamp', axis=1, inplace=True)
csv_tables['requester'].columns = ['requester_'+col for col in csv_tables['requester'].columns]

In [54]:
csv_tables['requester']

,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time
0,1,511,2.0,0,1956.0,2017,200,7,2,19,17,16,8.934592e+07
1,1,512,1.0,0,1990.0,2018,1,1,0,1,4,17,7.505029e+07
2,3,626,1.0,0,1985.0,2015,143,5,5,23,18,8,1.574260e+08
3,2,813,NaN,0,NaN,2015,320,11,0,16,15,47,1.421417e+08
4,1,1023,1.0,0,1987.0,2015,103,4,0,13,11,39,1.609054e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1745869,1,321431662,1.0,0,2015.0,2020,140,5,1,19,19,58,-8.779733e+04
1745870,1,321431877,2.0,0,1979.0,2020,140,5,1,19,19,46,-8.709054e+04
1745871,1,321432031,1.0,0,1959.0,2020,140,5,1,19,19,56,-8.768782e+04
1745872,1,321432602,1.0,0,1981.0,2020,140,5,1,19,20,0,-8.788295e+04


In [55]:
train_csv = train_csv.merge(csv_tables['requester'], how='left', left_on='requester', right_on='requester_id')

In [56]:
train_csv

,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time
0,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07
1,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06
2,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,48333290.0,FEDERAL,0.0,150,315,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07
3,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07
4,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713440,177596976,43,306416246,1275,4396466,8377002,4396466,2,2,63807360.0,4396466,SERVICE,12870937,0,NaN,210963.0,FEDERAL,6.0,8,104,3.089879e+06,1,177596976,2.0,0,1998.0,2020,140,5,1,19,13,7,-6.309740e+04
2713441,3663125,43,79455276,98,1956656,8377003,1956656,2,2,NaN,1956656,SERVICE,316134,0,NaN,48333290.0,FEDERAL,87.0,7,315,4.569523e+06,1,3663125,2.0,0,1975.0,2020,139,5,0,18,19,17,1.073953e+03
2713442,86143570,43,48742042,98,19146184,8377004,19146184,2,2,NaN,19146184,SERVICE,316134,0,NaN,48333290.0,FEDERAL,87.0,2,315,1.543392e+04,1,86143570,2.0,0,1958.0,2020,140,5,1,19,10,28,-5.356286e+04
2713443,138811055,43,1970511,491,1956423,8377005,1956423,10,5,NaN,1956423,SERVICE,316160,0,NaN,210889936.0,FEDERAL,90.0,3,282,6.374757e+06,1,138811055,1.0,0,1974.0,2020,139,5,0,18,19,35,3.300000e-01


#### cpgu_department

In [57]:
# Проанализировать title: кластеризовать/найти одинаковые/разбить по ведомствам

In [58]:
cpgu_dep_tfidf_vect = TfidfVectorizer(tokenizer=token_and_stem).fit_transform(csv_tables['cpgu_department'].title.values)

In [59]:
cpgu_clusters = AgglomerativeClustering(n_clusters=20, affinity='euclidean', linkage='average').fit_predict(cpgu_dep_tfidf_vect.toarray())

In [60]:
csv_tables['cpgu_department']['cluster'] = cpgu_clusters

In [61]:
csv_tables['cpgu_department'].drop('title', axis=1, inplace=True)

csv_tables['cpgu_department'].columns = ['cpgu_department_'+col for col in csv_tables['cpgu_department'].columns]

In [62]:
csv_tables['cpgu_department']

,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster
0,1147,NaN,10000001040,10000001040,180731844,4
1,1148,316185.0,10000001030,10000001030,180731844,0
2,1150,NaN,10000001265,10000001265,180731844,4
3,1151,1150.0,10001761399,10001761399,180731844,3
4,27786,NaN,3800000010000000833,3800000010000000833,180731844,4
...,...,...,...,...,...,...
192,191644781,NaN,38841641,38841641,180731844,6
193,191645089,NaN,38841642,38841642,180731844,6
194,208163832,NaN,38841643,38841643,180731844,6
195,287578549,NaN,3884162907,3884162907,180731844,2


In [63]:
train_csv = train_csv.merge(csv_tables['cpgu_department'], how='left', left_on='department_id', right_on='cpgu_department_id')

#### cpgu_mfc

In [64]:
csv_tables['cpgu_mfc'].drop(['name', 'full_name', 'address'], axis=1, inplace=True)
csv_tables['cpgu_mfc'].columns = ['cpgu_mfc_' + col for col in csv_tables['cpgu_mfc'].columns]

In [65]:
csv_tables['cpgu_mfc']

,cpgu_mfc_id,cpgu_mfc_code,cpgu_mfc_closed,cpgu_mfc_branch_id,cpgu_mfc_win_count,cpgu_mfc_qms_server_id,cpgu_mfc_deperatment_id,cpgu_mfc_office_type_id,cpgu_mfc_parent_id,cpgu_mfc_key_mfc,cpgu_mfc_calendar,cpgu_mfc_damask_office_id
0,159,69,0,12.0,12.0,12.0,5.0,1.0,159.0,shl,91802555,39.0
1,1003,40,0,1.0,30.0,1.0,1.0,1.0,1003.0,irk1,91802555,8.0
2,332483,24,0,13.0,6.0,13.0,5.0,1.0,332483.0,bkl,91793376,40.0
3,371402,26,0,15.0,3.0,15.0,6.0,1.0,371402.0,bnd,91793376,60.0
4,869529,64,0,16.0,7.0,16.0,6.0,1.0,869529.0,urd,91793376,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...
215,180669941,443,0,NaN,1.0,NaN,NaN,2.0,14855088.0,NaN,112362405,NaN
216,223567332,6012,0,NaN,1.0,NaN,NaN,2.0,10251121.0,NaN,112362405,NaN
217,223783662,645,0,NaN,1.0,NaN,NaN,2.0,869529.0,NaN,112362405,NaN
218,288545464,77,0,59.0,10.0,51.0,1.0,1.0,288545464.0,irk9,112362405,10.0


In [66]:
train_csv = train_csv.merge(csv_tables['cpgu_mfc'], how='left', left_on='mfc', right_on='cpgu_mfc_id')

In [67]:
train_csv

,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster,cpgu_mfc_id,cpgu_mfc_code,cpgu_mfc_closed,cpgu_mfc_branch_id,cpgu_mfc_win_count,cpgu_mfc_qms_server_id,cpgu_mfc_deperatment_id,cpgu_mfc_office_type_id,cpgu_mfc_parent_id,cpgu_mfc_key_mfc,cpgu_mfc_calendar,cpgu_mfc_damask_office_id
0,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN
1,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN
2,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,48333290.0,FEDERAL,0.0,150,315,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316134,316133.0,10000001055,10001742404,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN
3,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN
4,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713440,177596976,43,306416246,1275,4396466,8377002,4396466,2,2,63807360.0,4396466,SERVICE,12870937,0,NaN,210963.0,FEDERAL,6.0,8,104,3.089879e+06,1,177596976,2.0,0,1998.0,2020,140,5,1,19,13,7,-6.309740e+04,12870937,NaN,38131,10000682250,180731844,7,4396466,67,0,25.0,14.0,17.0,2.0,1.0,4396466.0,chr,91802555,24.0
2713441,3663125,43,79455276,98,1956656,8377003,1956656,2,2,NaN,1956656,SERVICE,316134,0,NaN,48333290.0,FEDERAL,87.0,7,315,4.569523e+06,1,3663125,2.0,0,1975.0,2020,139,5,0,18,19,17,1.073953e+03,316134,316133.0,10000001055,10001742404,180731844,0,1956656,23,0,8.0,13.0,8.0,2.0,1.0,1956656.0,ang2,91802555,22.0
2713442,86143570,43,48742042,98,19146184,8377004,19146184,2,2,NaN,19146184,SERVICE,316134,0,NaN,48333290.0,FEDERAL,87.0,2,315,1.543392e+04,1,86143570,2.0,0,1958.0,2020,140,5,1,19,10,28,-5.356286e+04,316134,316133.0,10000001055,10001742404,180731844,0,19146184,68,0,50.0,3.0,42.0,3.0,1.0,19146184.0,mag,91793376,54.0
2713443,138811055,43,1970511,491,1956423,8377005,1956423,10,5,NaN,1956423,SERVICE,316160,0,NaN,210889936.0,FEDERAL,90.0,3,282,6.374757e+06,1,138811055,1.0,0,1974.0,2020,139,5,0,18,19,35,3.300000e-01,316160,NaN,10000001086,10000001086,180731844,0,1956423,58,0,14.0,6.0,14.0,5.0,1.0,1956423.0,sld,91793376,41.0


#### cpgu_user

In [68]:
csv_tables['cpgu_user'].columns = ['cpgu_user_' + col for col in csv_tables['cpgu_user'].columns]

In [69]:
csv_tables['cpgu_user']

,cpgu_user_id,cpgu_user_auto_ping_queue
0,1,1
1,3,0
2,5,0
3,7,0
4,9,0
...,...,...
1904,319944271,1
1905,319945195,1
1906,319946930,1
1907,321054698,1


In [70]:
train_csv = train_csv.merge(csv_tables['cpgu_user'], how='left', left_on='cpgu_user', right_on='cpgu_user_id')

#### cpgu_service

In [71]:
csv_tables['cpgu_service'].drop([
    'description', 
    'department', 
    'eid', 
    'lid', 
    'parent_service_eid', 
    'parent_service_lid', 
    'frgu_service_id'
], axis=1, inplace=True)

csv_tables['cpgu_service'].columns = ['cpgu_service_' + col for col in csv_tables['cpgu_service'].columns]

In [72]:
csv_tables['cpgu_service']

,cpgu_service_id,cpgu_service_level,cpgu_service_is_not_render,cpgu_service_person,cpgu_service_sole,cpgu_service_legal
0,43,NaN,0,0,0,0
1,44,NaN,1,1,1,1
2,45,REGIONAL,1,1,1,1
3,46,NaN,1,1,1,1
4,47,NaN,1,1,1,1
...,...,...,...,...,...,...
157,113491294,REGIONAL,0,1,0,0
158,121350183,FEDERAL,1,1,0,0
159,136670703,OTHER,0,1,0,0
160,158094295,FEDERAL,1,1,0,0


In [73]:
train_csv = train_csv.merge(csv_tables['cpgu_service'], how='left', left_on='service', right_on='cpgu_service_id')

#### custom_service_info

In [74]:
# тут интересные название в 'eid',	'lid', 'parent_service_eid', 'parent_service_lid'

In [75]:
csv_tables['custom_service_info'].drop([
    'title',
    'eid',
    'lid',
    'parent_service_eid',
    'parent_service_lid',
    'frgu_service_id'
], axis=1, inplace=True)

csv_tables['custom_service_info'].columns = ['custom_service_info_' + col for col in csv_tables['custom_service_info'].columns]

In [76]:
csv_tables['custom_service_info']

,custom_service_info_id,custom_service_info_person,custom_service_info_sole,custom_service_info_legal,custom_service_info_group_title,custom_service_info_department,custom_service_info_level,custom_service_info_is_not_render,custom_service_info_deleted
0,27635,1,0,0,Другие услуги,316164.0,REGIONAL,0,0
1,27639,1,0,0,Другие услуги,316164.0,REGIONAL,0,0
2,27641,1,1,1,NaN,316134.0,FEDERAL,1,0
3,27694,1,1,1,NaN,9315713.0,FEDERAL,1,0
4,27696,1,1,1,NaN,9315713.0,FEDERAL,1,0
...,...,...,...,...,...,...,...,...,...
1692,320178499,1,0,0,NaN,316164.0,REGIONAL,0,0
1693,320247491,1,0,0,NaN,12870937.0,FEDERAL,0,0
1694,320248565,1,0,0,NaN,12870937.0,FEDERAL,0,0
1695,320249704,1,0,0,NaN,12870937.0,FEDERAL,0,0


In [77]:
train_csv = train_csv.merge(csv_tables['custom_service_info'], how='left', left_on='custom_service_id', right_on='custom_service_info_id')

In [78]:
train_csv

,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster,cpgu_mfc_id,cpgu_mfc_code,cpgu_mfc_closed,cpgu_mfc_branch_id,cpgu_mfc_win_count,cpgu_mfc_qms_server_id,cpgu_mfc_deperatment_id,cpgu_mfc_office_type_id,cpgu_mfc_parent_id,cpgu_mfc_key_mfc,cpgu_mfc_calendar,cpgu_mfc_damask_office_id,cpgu_user_id,cpgu_user_auto_ping_queue,cpgu_service_id,cpgu_service_level,cpgu_service_is_not_render,cpgu_service_person,cpgu_service_sole,cpgu_service_legal,custom_service_info_id,custom_service_info_person,custom_service_info_sole,custom_service_info_legal,custom_service_info_group_title,custom_service_info_department,custom_service_info_level,custom_service_info_is_not_render,custom_service_info_deleted
0,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN,193686,0,46880280,FEDERAL,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN,193686,0,43,NaN,0,0,0,0,329298.0,1.0,1.0,1.0,NaN,49801718.0,FEDERAL,0.0,0.0
2,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,48333290.0,FEDERAL,0.0,150,315,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316134,316133.0,10000001055,10001742404,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN,193686,0,43,NaN,0,0,0,0,48333290.0,1.0,1.0,1.0,NaN,316134.0,FEDERAL,0.0,0.0
3,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN,193686,0,46880280,FEDERAL,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,6559747,0,0,NaN,NaN,NaN,NaN,2.0,6559747.0,NaN,91802555,NaN,193686,0,43,NaN,0,0,0,0,329298.0,1.0,1.0,1.0,NaN,49801718.0,FEDERAL,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2713440,177596976,43,306416246,1275,4396466,8377002,4396466,2,2,63807360.0,4396466,SERVICE,12870937,0,NaN,210963.0,FEDERAL,6.0,8,104,3.089879e+06,1,177596976,2.0,0,1998.0,2020,140,5,1,19,13,7,-6.309740e+04,12870937,NaN,38131,10000682250,180731844,7,4396466,67,0,25.0,14.0,17.0,2.0,1.0,4396466.0,chr,91802555,24.0,306416246,1,43,NaN,0,0,0,0,210963.0,1.0,0.0,0.0,NaN,12870937.0,FEDERAL,0.0,0.0
2713441,3663125,43,79455276,98,1956656,8377003,1956656,2,2,NaN,1956656,SERVICE,316134,0,NaN,4

#### concat/merge

In [79]:
whole_dataset = pd.concat([
    train_csv,
    train_csv_datetimes,
    next_shifted_st,
    target,
], axis=1)

In [ ]:
whole_dataset

,requester,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster,...,time_since_last_act_order_date,time_since_last_act_issue_date,time_since_last_act_change_timestamp,time_since_last_act_close_date,order_date_year,order_date_day_of_year,order_date_month,order_date_weekday,order_date_day,order_date_hour,order_date_minute,order_date_ms_since_last_time,issue_date_year,issue_date_day_of_year,issue_date_month,issue_date_weekday,issue_date_day,issue_date_hour,issue_date_minute,issue_date_ms_since_last_time,change_timestamp_year,change_timestamp_day_of_year,change_timestamp_month,change_timestamp_weekday,change_timestamp_day,change_timestamp_hour,change_timestamp_minute,change_timestamp_ms_since_last_time,close_date_year,close_date_day_of_year,close_date_month,close_date_weekday,close_date_day,close_date_hour,close_date_minute,close_date_ms_since_last_time,waiting_time_ord_cls,previous_service_title,previous_text_cluster,target
0,7184765,46880280,193686,491,6559747,5639098,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,...,NaN,NaN,NaN,NaN,2019,6,1,6,6,16,42,4.303757e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,2019,6,1,6,6,16,49,4.303718e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,-60150.203,NaN,NaN,134.0
1,4579720,43,193686,815,6559747,5639099,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,...,NaN,NaN,NaN,NaN,2019,6,1,6,6,17,40,4.303408e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,2019,6,1,6,6,17,42,4.303398e+07,2019.0,6.0,1.0,6.0,6.0,0.0,0.0,4.309773e+07,-63647.163,NaN,NaN,815.0
2,7184765,43,193686,134,6559747,5639100,6559747,10,1853152,NaN,6559747,SERVICE,316134,0,NaN,48333290.0,FEDERAL,0.0,150,315,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316134,316133.0,10000001055,10001742404,180731844,0,...,1.645387e+05,172800.0,1.672823e+05,172800.0,2019,8,1,1,8,14,24,4.287304e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,17,4.286990e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-51888.943,491.0,282.0,491.0
3,7184765,46880280,193686,491,6559747,5639101,6559747,10,5,NaN,6559747,SERVICE,316160,0,NaN,NaN,FEDERAL,90.0,150,282,2.021659e+05,1,7184765,NaN,0,1970.0,2020,6,1,0,6,19,23,1.149189e+07,316160,NaN,10000001086,10000001086,180731844,0,...,3.302750e+03,0.0,2.016300e+02,0.0,2019,8,1,1,8,15,19,4.286973e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,20,4.286969e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-55191.693,134.0,315.0,1169.0
4,4579720,43,193686,815,6559747,5639102,6559747,10,5,NaN,6559747,SERVICE,49801718,0,NaN,329298.0,FEDERAL,0.0,3,181,1.344898e+07,1,4579720,1.0,0,1985.0,2020,108,4,4,17,17,12,2.686959e+06,49801718,NaN,38841632,10001095257,180731844,9,...,1.648503e+05,172800.0,1.648454e+05,172800.0,2019,8,1,1,8,15,28,4.286923e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,2019,8,1,1,8,15,29,4.286913e+07,2019.0,8.0,1.0,1.0,8.0,0.0,0.0,4.292493e+07,-55697.503,815.0,181.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Тут общая предобработка заканчивается и начинается подготовка трейна и теста

In [ ]:
train_dataset = whole_dataset.drop(whole_dataset[whole_dataset.target.isna()].index)

In [ ]:
last_index_for_each_req_train = train_dataset.reset_index().groupby('requester')['index'].max()
unique_train_dataset = whole_dataset.iloc[last_index_for_each_req_train.values].set_index('requester')

In [ ]:
# bow_cluster_count_train = train_dataset.reset_index().pivot_table(
#     index='requester', 
#     columns='emb_item_cluster',
#     values='index', 
#     aggfunc='count',
# )
# bow_cluster_count_train.columns = ['bow_cluster_count_'+str(col) for col in bow_cluster_count_train.columns]
# bow_cluster_count_train

In [ ]:
# unique_train_dataset = pd.concat([
#     unique_train_dataset,
#     bow_cluster_count_train,                              
# ], axis=1)

In [ ]:
unique_train_dataset['n_visits'] -= 1

In [ ]:
# unique_train_dataset = unique_train_dataset.merge(user_embeddings_df, how='left', left_index=True, right_index=True)
# unique_train_dataset = unique_train_dataset.merge(lfm_preds_df, how='left', left_index=True, right_index=True)

In [ ]:
unique_train_dataset

,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster,cpgu_mfc_id,...,time_since_last_act_order_date,time_since_last_act_issue_date,time_since_last_act_change_timestamp,time_since_last_act_close_date,order_date_year,order_date_day_of_year,order_date_month,order_date_weekday,order_date_day,order_date_hour,order_date_minute,order_date_ms_since_last_time,issue_date_year,issue_date_day_of_year,issue_date_month,issue_date_weekday,issue_date_day,issue_date_hour,issue_date_minute,issue_date_ms_since_last_time,change_timestamp_year,change_timestamp_day_of_year,change_timestamp_month,change_timestamp_weekday,change_timestamp_day,change_timestamp_hour,change_timestamp_minute,change_timestamp_ms_since_last_time,close_date_year,close_date_day_of_year,close_date_month,close_date_weekday,close_date_day,close_date_hour,close_date_minute,close_date_ms_since_last_time,waiting_time_ord_cls,previous_service_title,previous_text_cluster,target
requester,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1651,43,19571394,339,1788235,6667186,1788235,10,5,NaN,1788235,CONSULTATION,12870937,0,NaN,1932695.0,FEDERAL,79.0,5,113,4.940439e+06,1,1651,2.0,0,1990.0,2019,137,5,4,17,15,28,3.172364e+07,12870937,NaN,38131,10000682250,180731844,7,1788235,...,4.486460e+06,NaN,4.484494e+06,NaN,2019,189,7,0,8,13,14,2.723887e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,189,7,0,8,13,14,2.723887e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.0,74.0,452.0
5432,43,19913325,604,159,7115618,159,10,5,NaN,159,CONSULTATION,316164,0,NaN,459072.0,REGIONAL,30.0,2,4,6.654813e+06,1,5432,2.0,0,1988.0,2019,256,9,4,13,12,36,2.145234e+07,316164,NaN,3800000010000000672,3800000010000000672,180731844,4,159,...,1.988788e+07,NaN,1.839395e+07,NaN,2019,255,9,3,12,15,11,2.152944e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,255,9,3,12,15,11,2.152944e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,39.0,604.0
5439,43,219411,134,84443014,5911068,84443014,10,5,NaN,84443014,CONSULTATION,316134,0,NaN,48333290.0,FEDERAL,0.0,1,315,1.224200e+01,1,5439,1.0,0,1960.0,2019,52,2,3,21,13,26,3.907491e+07,316134,316133.0,10000001055,10001742404,180731844,0,84443014,...,NaN,NaN,NaN,NaN,2019,52,2,3,21,13,26,3.907494e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,52,2,3,21,13,27,3.907490e+07,2019.0,52.0,2.0,3.0,21.0,0.0,0.0,3.912333e+07,-48388.823,NaN,NaN,524.0
5471,43,159970,98,159,7632966,159,10,5,NaN,159,SERVICE,316134,0,NaN,48333290.0,FEDERAL,87.0,3,315,2.800630e+06,1,5471,2.0,0,1939.0,2019,361,12,4,27,10,13,1.238889e+07,316134,316133.0,10000001055,10001742404,180731844,0,159,...,7.157802e+06,NaN,5.945836e+06,5961600.0,2019,340,12,4,6,10,13,1.420333e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,343,12,0,9,9,13,1.394771e+07,2019.0,343.0,12.0,0.0,9.0,0.0,0.0,1.398093e+07,222402.560,134.0,315.0,98.0
5477,43,159970,548,159,6950085,159,10,5,240217606.0,159,SERVICE,316132,0,NaN,2405312.0,FEDERAL,18.0,3,40,2.375728e+06,1,5477,2.0,0,1974.0,2019,350,12,0,16,17,22,1.331355e+07,316132,316131.0,10000001764,10001903047,180731844,7,159,...,4.214207e+05,NaN,-6.820381e+05,-691200.0,2019,232,8,1,20,12,6,2.352771e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,241,8,3,29,15,25,2.273818e+07,2019.0,241.0,8.0,3.0,29.0,0.0,0.0,2.279373e+07,733985.923,604.0,4.0,870.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

---

In [ ]:
test_dataset_ids = whole_dataset.reset_index().groupby('requester')['index'].max().values

In [ ]:
presence_test_req = (
    set(whole_dataset.loc[test_dataset_ids, 'requester'].values)
    .intersection(test_ids.values)
)
absence_test_req = set(test_ids.values) - presence_test_req

len(presence_test_req), len(absence_test_req)

(44443, 1)

In [ ]:
test_dataset = whole_dataset.loc[test_dataset_ids].set_index('requester').loc[presence_test_req].reset_index()
test_dataset.drop(['target'], axis=1, inplace=True)

In [ ]:
# bow_cluster_count_test = test_dataset.reset_index().pivot_table(
#     index='requester', 
#     columns='emb_item_cluster',
#     values='index', 
#     aggfunc='count',
# )
# bow_cluster_count_test.columns = ['bow_cluster_count_'+str(col) for col in bow_cluster_count_test.columns]

In [ ]:
# test_dataset = pd.concat([
#     test_dataset.set_index('requester'), 
#     bow_cluster_count_test,                              
# ], axis=1)

In [ ]:
# test_dataset = test_dataset.merge(user_embeddings_df, how='left', left_on='requester', right_index=True)
# test_dataset = test_dataset.merge(lfm_preds_df, how='left', left_index=True, right_index=True)

In [ ]:
test_dataset.set_index('requester', inplace=True)

In [ ]:
# columns_to_add = list(set(unique_train_dataset.columns) - set(test_dataset.columns))
# common_columns = list(set(unique_train_dataset.columns).intersection(test_dataset.columns))
# test_dataset = pd.concat([
#     test_dataset[common_columns], 
#     pd.DataFrame(columns=columns_to_add)
# ], axis=1).drop('target', axis=1)

In [ ]:
test_dataset

,service,cpgu_user,service_title,receipt_mfc,order_number,mfc,internal_status,external_status,sub_department,creation_mfc,order_type,department_id,deleted,deleter_fk,custom_service_id,service_level,emb_item_cluster,n_visits,text_cluster,visit_freq,requester_requester_type,requester_id,requester_gender,requester_deleted,requester_birth_year,requester_edit_timestamp_year,requester_edit_timestamp_day_of_year,requester_edit_timestamp_month,requester_edit_timestamp_weekday,requester_edit_timestamp_day,requester_edit_timestamp_hour,requester_edit_timestamp_minute,requester_edit_timestamp_ms_since_last_time,cpgu_department_id,cpgu_department_parent,cpgu_department_department_id,cpgu_department_code,cpgu_department_calendar,cpgu_department_cluster,cpgu_mfc_id,...,custom_service_info_deleted,time_since_last_act_order_date,time_since_last_act_issue_date,time_since_last_act_change_timestamp,time_since_last_act_close_date,order_date_year,order_date_day_of_year,order_date_month,order_date_weekday,order_date_day,order_date_hour,order_date_minute,order_date_ms_since_last_time,issue_date_year,issue_date_day_of_year,issue_date_month,issue_date_weekday,issue_date_day,issue_date_hour,issue_date_minute,issue_date_ms_since_last_time,change_timestamp_year,change_timestamp_day_of_year,change_timestamp_month,change_timestamp_weekday,change_timestamp_day,change_timestamp_hour,change_timestamp_minute,change_timestamp_ms_since_last_time,close_date_year,close_date_day_of_year,close_date_month,close_date_weekday,close_date_day,close_date_hour,close_date_minute,close_date_ms_since_last_time,waiting_time_ord_cls,previous_service_title,previous_text_cluster
requester,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
19529735,43,101142159,4,8433794,6000286,8433794,10,1853152,NaN,8433794,SERVICE,9315713,0,NaN,48333336.0,FEDERAL,51.0,1,39,0.000000e+00,1,19529735,1.0,0,1981.0,2019,70,3,0,11,14,40,3.751530e+07,9315713,NaN,10001742404,10001742620,180731844,0,8433794,...,0.0,NaN,NaN,NaN,NaN,2019,70,3,0,11,14,34,3.751566e+07,2019.0,77.0,3.0,0.0,18.0,0.0,0.0,3.696333e+07,2019,77,3,0,18,13,6,3.691614e+07,2019.0,77.0,3.0,0.0,18.0,0.0,0.0,3.696333e+07,552337.287,NaN,NaN
10878986,65129676,213852346,1169,8951175,7223868,8951175,10,1853152,NaN,8951175,SERVICE,49801718,0,NaN,NaN,FEDERAL,33.0,2,141,4.920799e+06,1,10878986,2.0,0,1967.0,2019,280,10,0,7,12,14,1.938008e+07,49801718,NaN,38841632,10001095257,180731844,9,8951175,...,NaN,9841598.973,NaN,1.416986e+07,NaN,2019,273,9,0,30,11,51,1.998625e+07,2019.0,323.0,11.0,1.0,19.0,0.0,0.0,1.570893e+07,2019,323,11,1,19,14,9,1.565798e+07,2019.0,323.0,11.0,1.0,19.0,0.0,0.0,1.570893e+07,4277321.460,1259.0,36.0
260440076,43,16165165,134,14857421,7155867,14857421,10,1853152,NaN,14857421,SERVICE,316134,0,NaN,48333290.0,FEDERAL,0.0,1,315,0.000000e+00,1,260440076,2.0,0,2014.0,2019,262,9,3,19,10,11,2.094264e+07,316134,316133.0,10000001055,10001742404,180731844,0,14857421,...,0.0,NaN,NaN,NaN,NaN,2019,262,9,3,19,9,48,2.094400e+07,2019.0,276.0,10.0,3.0,3.0,0.0,0.0,1.976973e+07,2019,276,10,3,3,12,22,1.972519e+07,2019.0,276.0,10.0,3.0,3.0,0.0,0.0,1.976973e+07,1174275.347,NaN,NaN
30408720,43,872171,178,869529,7374064,869529,1853149,5,NaN,869529,SERVICE,316129,0,NaN,179048940.0,FEDERAL,91.0,1,189,0.000000e+00,1,30408720,1.0,0,1987.0,2019,297,10,3,24,10,30,1.791749e+07,316129,316128.0,10000001169,10003716045,180731844,0,869529,...,0.0,NaN,NaN,NaN,NaN,2019,296,10,2,23,15,7,1.798728e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019,352,12,2,18,4,15,1.318797e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277741586,43,51040375,573,1003,7502986,1003,10,1853152,NaN,1003,SERVICE,49801718,0,NaN,50285368.0,FEDERAL,81.0,1,96,0.000000e+00,1,277741586,1.0,0,1965.0,2019,318,11,3,14,19,52,1.606935e+07,49801718,NaN,38841632,10001095257,180731844,9,1003,...,0.0,NaN,NaN,NaN,NaN,2019,318,11,3,14,19,30,1.607067e+07,2019.0,354.0,12.0,4.0,20.0,0.0,0.0,1.303053e+07,2019,354,12,4,20,13,6,1.298332e+07,2019.0,354.0,12.0,4.0,20.0,0.0,0.0,1.303053e+07,30

In [ ]:
unique_train_dataset.to_csv(global_path+'unique_train_dataset_3767.csv', chunksize=10000)

In [ ]:
test_dataset.to_csv(global_path+'test_dataset_3767.csv', chunksize=10000)

# Model & submit

---

In [83]:
loaded_train_dataset = pd.read_csv(global_path+'unique_train_dataset_3767.csv')
loaded_test_dataset = pd.read_csv(global_path+'test_dataset_3767.csv')

In [ ]:
# cols = loaded_test_dataset.columns.values
# cols[0] = 'requester'
# loaded_test_dataset.columns = cols

In [ ]:
# columns_to_drop = [col for col in loaded_test_dataset.columns if (
#     col.startswith('user_embedding') or 
#     col.startswith('lfm_pred') or 
#     col.startswith('bow')
# )]

# loaded_test_dataset.drop(columns_to_drop, axis=1, inplace=True)
# loaded_train_dataset.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# loaded_test_dataset = loaded_test_dataset[loaded_train_dataset.drop(['target'], axis=1).columns.values]

In [84]:
for feat in tqdm(categorical_features):
    loaded_train_dataset[feat] = loaded_train_dataset[feat].astype(str)
    loaded_test_dataset[feat] = loaded_test_dataset[feat].astype(str)

loaded_train_dataset[categorical_features] = loaded_train_dataset[categorical_features].fillna('nan_value')
loaded_test_dataset[categorical_features] = loaded_test_dataset[categorical_features].fillna('nan_value')

In [ ]:
# для LightGBM
train_test_dataset = pd.concat([
    loaded_train_dataset[categorical_features],
    loaded_test_dataset[categorical_features]
])

for feat in tqdm(categorical_features):
    le = LabelEncoder().fit(train_test_dataset[feat])
    
    loaded_train_dataset[feat] = pd.to_numeric(le.transform(loaded_train_dataset[feat]))
    loaded_test_dataset[feat] = pd.to_numeric(le.transform(loaded_test_dataset[feat]))

In [16]:
single_classes = loaded_train_dataset['target'].value_counts() == 1
single_classes = single_classes[single_classes].index.tolist()
len(single_classes)

142

In [17]:
single_classes_ind = loaded_train_dataset[loaded_train_dataset['target'].isin(single_classes)].index
loaded_train_dataset.drop(single_classes_ind, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    loaded_train_dataset.drop('target', axis=1), 
    loaded_train_dataset['target'], 
    test_size=0.15, 
    random_state=1234,
    stratify=loaded_train_dataset['target']
)

#### catboost

In [ ]:
# мода, рнн, как получать нормальные эмбеддинги через rs? 
catboost_params_l2 = {
    'task_type': 'GPU',
    'eval_metric': 'Accuracy',
    'random_state': 7492,
    'verbose': 1,
    'loss_function': 'MultiClass',
    'iterations': 1500,
    'border_count': 32,
    'cat_features': categorical_features,
    'max_ctr_complexity': 5,
    'depth': 6,
    'l2_leaf_reg': 1,

    'auto_class_weights': 'SqrtBalanced'
    'model_size_reg': 0.1

    'train_dir': global_path+'model_info/',
    'save_snapshot': True,
    'snapshot_file': global_path+'catboost_snapshots/catboost_model_wo_bow_ue_lfmpred_sqrt_5.snap',
    'snapshot_interval': 600,
}

In [ ]:
model_l2 = CatBoostClassifier(**catboost_params_l2)

In [ ]:
model_l2.fit(
    X_train, 
    y_train, 
    eval_set=(X_test, y_test)
)

1150:	learn: 0.5311437	test: 0.4999709	best: 0.5000000 (1146)	total: 8h 28m 55s	remaining: 2h 37m 38s
1151:	learn: 0.5311771	test: 0.4999126	best: 0.5000000 (1146)	total: 8h 29m 22s	remaining: 2h 36m 23s
1152:	learn: 0.5311951	test: 0.4999563	best: 0.5000000 (1146)	total: 8h 29m 49s	remaining: 2h 35m 51s
1153:	learn: 0.5312567	test: 0.4999563	best: 0.5000000 (1146)	total: 8h 30m 15s	remaining: 2h 34m 46s
1154:	learn: 0.5313081	test: 0.4999126	best: 0.5000000 (1146)	total: 8h 30m 42s	remaining: 2h 34m 2s
1155:	learn: 0.5313981	test: 0.4998981	best: 0.5000000 (1146)	total: 8h 31m 9s	remaining: 2h 33m 33s
1156:	learn: 0.5314109	test: 0.4999418	best: 0.5000000 (1146)	total: 8h 31m 36s	remaining: 2h 33m 12s
1157:	learn: 0.5314186	test: 0.4999709	best: 0.5000000 (1146)	total: 8h 32m 2s	remaining: 2h 32m 36s
1158:	learn: 0.5314212	test: 0.4999709	best: 0.5000000 (1146)	total: 8h 32m 29s	remaining: 2h 31m 57s
1159:	learn: 0.5314572	test: 0.5000146	best: 0.5000146 (1159)	total: 8h 32m 55s	remai

In [ ]:
#pickle.dump(model_l2, open(global_path+'full_ds_1500', 'wb'))

In [85]:
model_l2 = pickle.load(open(global_path+'full_ds_1500', 'rb'))

In [57]:
pd.set_option('display.max_rows', 200)

In [58]:
pd.Series(model_l2.feature_importances_, model_l2.feature_names_).sort_values(ascending=False)

service_title                                  15.468639
requester_edit_timestamp_ms_since_last_time    13.935603
previous_service_title                          5.499386
custom_service_id                               4.839353
visit_freq                                      4.481340
cpgu_mfc_deperatment_id                         3.505376
emb_item_cluster                                3.504310
requester_birth_year                            3.168499
text_cluster                                    2.536501
cpgu_user                                       2.282033
requester_edit_timestamp_day_of_year            2.062367
previous_text_cluster                           1.628777
cpgu_mfc_parent_id                              1.554047
time_since_last_act_order_date                  1.483044
custom_service_info_id                          1.469992
requester_edit_timestamp_hour                   1.389126
cpgu_mfc_win_count                              1.281595
cpgu_department_cluster        

In [100]:
pred_probas = model_l2.predict_proba(loaded_test_dataset)

In [86]:
preds = model_l2.predict(loaded_test_dataset)

In [89]:
preds[loaded_test_dataset.n_visits == 1] = [1309]

#### tensorboard

In [ ]:
import tensorflow as tf
%load_ext tensorboard

In [ ]:
#!rm -rf '/content/drive/My Drive/ML/Выявление закономерностей на основе данных МФЦ/model_info/'
%tensorboard --logdir '/content/drive/My Drive/ML/Выявление закономерностей на основе данных МФЦ/model_info/'

## Prediction

In [ ]:
preds = []

for i, proba in enumerate(tqdm(pred_probas)):
    if max(proba) > 0.75:
        if (model_l2.classes_[proba.argmax()] == 1259 and max(proba) < 0.8):
            if proba[idx_98_cls] > proba[idx_134_cls]:
                preds.append(model_l2.classes_[idx_98_cls])
            else:
                preds.append(model_l2.classes_[idx_134_cls])
        else:
            preds.append(model_l2.classes_[proba.argmax()])
    else:
        preds.append(1309)

preds = np.array(preds)

In [91]:
pd.Series(preds.flatten()).value_counts().iloc[:15]

1309.0    30746
98.0       1873
134.0      1813
155.0       664
207.0       646
907.0       606
603.0       606
1205.0      559
604.0       549
901.0       512
1020.0      389
4.0         357
1259.0      306
966.0       242
1220.0      232
dtype: int64

In [ ]:
id_to_desc.loc[1259]

1               Регистрация пользователя ЕСИА
text_cluster                               36
Name: 1259, dtype: object

In [ ]:
id_to_desc[id_to_desc.text_cluster == 36]

,1,text_cluster
19,Регистрация пользователя ЕСИА (СМЭВ 2),36
218,Регистрация ЕСИА (СМЭВ 3),36
907,Восстановление пароля пользователя ЕСИА,36
1170,Получение статуса заявки на регистрацию ЕСИА,36
1259,Регистрация пользователя ЕСИА,36


---

In [92]:
preds_df = pd.DataFrame(
    data=preds.flatten(), 
    index=loaded_test_dataset['requester'],
    columns=['service_title'],
    dtype=int
)

In [93]:
preds_df

,service_title
requester,
19529735,1309
10878986,901
260440076,1309
30408720,1309
277741586,1309
...,...
238682101,1309
315490293,1309
219152375,907


In [94]:
sample_subm = pd.read_csv(irkutsk_ds_path+'sample_submission.csv')
sample_subm

,requester,service_title
0,83029018,1
1,123998836,1
2,184301121,1
3,54344611,1
4,184299902,1
...,...,...
44439,27213855,1
44440,64899947,1
44441,23695212,1
44442,72707269,1


In [95]:
set(sample_subm.requester) - set(loaded_test_dataset.requester.astype(int))

{67480424}

In [96]:
preds_df.loc[67480424, 'service_title'] = 1309
preds_df

,service_title
requester,
19529735,1309.0
10878986,901.0
260440076,1309.0
30408720,1309.0
277741586,1309.0
...,...
315490293,1309.0
219152375,907.0
79036406,1309.0


In [97]:
preds_df.reset_index(inplace=True)
preds_df.requester = preds_df.requester.astype(int)
preds_df.set_index('requester', inplace=True)

preds_df.service_title = preds_df.service_title.astype(int)

In [98]:
preds_df.loc[sample_subm['requester']].reset_index().to_csv(global_path+'subm.csv')

In [99]:
preds_df.loc[sample_subm['requester']].reset_index()

,requester,service_title
0,83029018,1220
1,123998836,1018
2,184301121,134
3,54344611,1259
4,184299902,1309
...,...,...
44439,27213855,1309
44440,64899947,1309
44441,23695212,1309
44442,72707269,1309
